In [73]:
import pandas as pd
import json
import os
import re

path = 'C:\\Users\\natha\\PycharmProjects\\gambling_premises_uk\\config\\2024_premises-licence-register_metadata.json'

with open(path, 'r') as file:
    data = json.load(file)

data.keys()

dict_keys(['imputation_rules', 'deduplication_rules', 'replacement_rules'])

In [74]:
path ='C:\\Users\\natha\\.cache\\kagglehub\\datasets\\nathanhg\\uk-gam-datasets\\versions\\1'
premises = '2024_premises-licence-register.csv'

# Read files into memory 

premises_df = pd.read_csv(os.path.join(path,premises))

Check column count

In [75]:
df = premises_df
# log discrepencies
columns_included = len(data['imputation_rules'].keys())
columns_expected = len(df.columns)
if columns_expected - columns_expected != 0:
    print('Analyst has not included all columns in imputation rules')
else:
    print('All columns included in imputation rules')

All columns included in imputation rules


In [76]:
def header_standardization(df):
    """
    Standardizes column names based on BigQuery lexical structure and syntax:
    https://cloud.google.com/bigquery/docs/reference/standard-sql/lexical#column_names
    Column names are changed if they violate the rules.
    
    Parameters:
        column_names (list): The list of original column names to standardize.
    
    Returns:
        list: The list of standardized column names.
    """
    column_names = df.columns
    # Define banned prefixes
    banned_prefixes = ['_TABLE_', '_FILE_', '_PARTITION', '_ROW_TIMESTAMP', '__ROOT__', '_COLIDENTIFIER']
    banned_prefixes = [prefix.lower() for prefix in banned_prefixes]

    # Initialize storage for validated column names and duplicates
    validated = []
    seen = set()

    for i, name in enumerate(column_names):
        # Ensure name is a string and convert to lowercase
        name = str(name).strip().lower()

        # Replace spaces with underscores and keep valid characters only
        name = re.sub(r'[^a-z0-9_]', '', name.replace(' ', '_'))

        # Enforce max length of 300 characters
        name = name[:300]

        # Handle banned prefixes
        for prefix in banned_prefixes:
            if name.startswith(prefix):
                name = name.replace(prefix, 'default', 1)

        # Prepend "column" if the name starts with a digit or is empty
        if not name or name[0].isdigit():
            name = f"column_{name}"

        # Resolve duplicates by appending a counter
        original_name = name
        counter = 1
        while name in seen:
            name = f"{original_name}_{counter}"
            counter += 1

        # Add the standardized name to the result and mark it as seen
        validated.append(name)
        seen.add(name)
    df.columns = validated
    return None


In [77]:
import pandas as pd

def clean_and_impute_data(df, metadata):
    """
    Cleans and imputes data based on provided metadata.

    Args:
        df (pd.DataFrame): The DataFrame to process.
        metadata (dict): Metadata for cleaning, specifying column types and imputation strategies.
    
    Returns:
        pd.DataFrame: The cleaned and imputed DataFrame.
    """
    for column, rules in metadata.items():
        col_type = rules.get("type")
        impute_method = rules.get("method")
        reference_column = rules.get("reference_column")

        # Handle data type conversions based on rules
        if col_type == "categorical":
            if df[column].dtype != 'object':  # Only convert if not already a string
                df[column] = df[column].astype(str)
                print(f'{column} converted to string')
        elif col_type == "numeric":
            if df[column].dtype != 'float64':  # Only convert if not already numeric
                df[column] = pd.to_numeric(df[column], errors='coerce')
                print(f'{column} converted to numeric')
        elif col_type == "datetime":
            if df[column].dtype != 'datetime64[ns]':  # Check if already datetime
                df[column] = pd.to_datetime(df[column], format='%Y-%m-%d %H:%M:%S', errors='coerce')
                print(f'{column} converted to datetime')
        elif col_type == "date":
            if not pd.api.types.is_datetime64_any_dtype(df[column]):  # Check for datetime type first
                df[column] = pd.to_datetime(df[column], format='%Y-%m-%d', errors='coerce').dt.date
                print(f'{column} converted to date')

        # Impute missing values based on specified method
        if impute_method == "NMAR":
            df.fillna({column:"Missing"}, inplace=True)
            print(f'Filled nulls in {column} with "Missing"')
        elif impute_method == "MAR":
            if reference_column:
                # Use the reference column to fill missing values
                df[column] = df.groupby(reference_column)[column].transform(
                    lambda x: x.ffill().bfill() if x.isnull().any() else x
                )
                print(f'Filled nulls in {column} using forward/backward fill based on {reference_column}')
            else:
                print(f'Skipping imputation for {column} due to missing reference_column')
        else:
            print(f'No imputation strategy specified for {column}')
    
    return df



In [78]:
header_standardization(df)

In [79]:
df[df['address_line_2'].isna()].shape[0]

20

In [80]:
clean_and_impute_data(df,data['imputation_rules'])
df[df['address_line_2'].isna()].shape[0]

account_number converted to numeric
Filled nulls in account_number using forward/backward fill based on account_name
Filled nulls in account_name using forward/backward fill based on account_number
Filled nulls in premises_activity with "Missing"
Filled nulls in local_authority with "Missing"
Filled nulls in address_line_1 with "Missing"
Filled nulls in address_line_2 with "Missing"
Filled nulls in city using forward/backward fill based on account_number
Filled nulls in postcode using forward/backward fill based on account_number


0

In [81]:
data['deduplication_rules']

{'deduplicate': True}

In [82]:
df.shape[0]

2660

In [83]:
def deduplicate(df, metadata):
    """
    Handles deduplication for the entire dataset based on metadata.
    Checks for duplicates before attempting to drop any.
    
    Args:
        df (pd.DataFrame): Input DataFrame.
        metadata (dict): Metadata defining deduplication rules.
    
    Returns:
        pd.DataFrame: DataFrame after applying deduplication rules.
    """
    deduplication_required = metadata.get("deduplication_rules", {}).get("deduplicate", False)
    if deduplication_required:
        # Check if there are any duplicate rows in the DataFrame
        duplicate_count = df.duplicated().sum()
        
        if duplicate_count > 0:
            # If duplicates are found, drop them
            df = df.drop_duplicates(keep="first")
            print(f"Dropped {duplicate_count} duplicate rows.")
    else:
        print("No duplicates to drop.")
    
    return None

In [84]:
deduplicate(df,data['deduplication_rules'])

No duplicates to drop.


In [85]:
data['replacement_rules']

{'premises_activity': {'Casino 2005': 'Casino'},
 'address_line_1': {"caesar's palace": 'caesars palace',
  'boylesport': 'boylesports',
  'coastal amusements limited': 'coastal amusements',
  'carousel amusements limited': 'carousel amusements',
  'blue anchor leisure limited': 'blue anchor leisure',
  'betting shop operations limited': 'betting shop operations',
  'betextra': 'bet extra',
  'betszone': 'betzone',
  'a & s leisure group limited': 'a & s leisure group',
  'a & s leisure': 'a & s leisure group',
  'cashino gaming limited': 'cashino gaming'},
 'address_line_2': {'coral island, unit 2 promenade': 'coral island, unit 2, promenade',
  '98a -99 high street, gorleston': '98a-99 high street, gorleston',
  '36 -42 marine terrace ': '36-42 marine terrace'},
 'city': {'Middlesborough': 'Middlesbrough',
  'Burton-on-trent': 'Burton-upon-trent',
  'Blaydon-upon-tyne': 'Blaydon-on-tyne',
  'Bury': 'Bury st. edmunds',
  'Lytham': 'Lytham st. annes',
  'Richmond': 'Richmond-upon-thame

In [86]:
def clean_strings(df, metadata):
   
    for column, replacements in metadata.items():
        if column not in df.columns:
            continue
        df[column] = df[column].astype(str).apply(lambda x: x.strip().lower())
        # Apply replacement rules and print details
        for old_value, new_value in replacements.items():
            old_value = old_value.strip().lower()
            new_value = new_value.strip().lower()
            occurrences = (df[column] == old_value).sum()
            if occurrences > 0:
                print(f'{column}: Replacing {occurrences} occurrences of "{old_value}" with "{new_value.title()}".')
                df[column] = df[column].replace({old_value.lower(): new_value.title()}, regex=True)
            else: 
                print(f'{column} has {occurrences} occurances of {old_value.lower()}')
        # Confirm number of replacements
    print('Completed all replacements.')

    return None

In [87]:
clean_strings(df,data['replacement_rules'])


premises_activity: Replacing 1 occurrences of "casino 2005" with "Casino".
address_line_1: Replacing 1 occurrences of "caesar's palace" with "Caesars Palace".
address_line_1: Replacing 1 occurrences of "boylesport" with "Boylesports".
address_line_1: Replacing 10 occurrences of "coastal amusements limited" with "Coastal Amusements".
address_line_1: Replacing 1 occurrences of "carousel amusements limited" with "Carousel Amusements".
address_line_1: Replacing 1 occurrences of "blue anchor leisure limited" with "Blue Anchor Leisure".
address_line_1: Replacing 9 occurrences of "betting shop operations limited" with "Betting Shop Operations".
address_line_1: Replacing 1 occurrences of "betextra" with "Bet Extra".
address_line_1: Replacing 1 occurrences of "betszone" with "Betzone".
address_line_1: Replacing 1 occurrences of "a & s leisure group limited" with "A & S Leisure Group".
address_line_1: Replacing 4 occurrences of "a & s leisure" with "A & S Leisure Group".
address_line_1: Replacin